Developed by Megan Renz for Cornell Physics Labs.

In [1]:
import sys
if sys.version_info[0] < 3:
    print("You appear to be using python 2 instead of python 3.  It is possible not all of the functionality will work.")

%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import *
import csv
from scipy.optimize import minimize, rosen, rosen_der, curve_fit
from matplotlib.widgets import TextBox
import pandas as pd 

def chiSquared2(args,x, y, dy, f ):
    '''Function Chi-Squared.  
    x, y and dy are numpy arrays, referring to x, y and the uncertainty in y respectively.
    f is the function we are fitting. 
    args are the arguments of the function we have fit.  
    '''
    return 1/(len(x)-len(args))*np.sum((f(x, args)-y)**2/dy**2)
def chiSquared(x, y, dy, f, args):
    '''Function Chi-Squared.  
    x, y and dy are numpy arrays, referring to x, y and the uncertainty in y respectively.
    f is the function we are fitting. 
    args are the arguments of the function we have fit.  
    '''
    return 1/(len(x)-len(args))*np.sum((f(x, args)-y)**2/dy**2)

def poly(x, args):
    '''
    returns the value of the polynomial sum (x**i*args[i])
    '''
    total=x**0*args[0]
    for i in range(1,len(args)):
        total+=x**i*args[i]
    return total
def linear(x,args):
    '''
    A special case of Poly.  
    '''
    return args[0]+x*args[1]
def linearFit(x, a,b):
    return b*x+a


def autoFit(x=[], y=[], dy=[], title="Use title= in your call", xaxis="Use xaxis= in your call", yaxis="Use yaxis= in your call", function=linear, guess=[0,1], n=10, movePoints=False, sliders=False, path=''):
    '''
    Function:  autoFit
    
    This function is meant to be very all-purpose for fitting your data using chi-squared.  
    
    In order to call it, use autoFit(put your arguments here).  
    
    There are a number of options you can use to input your data.
    
    1.  If you call the function with autoFit(x=1-d vector containing the x values of your data, y=1-d array containing the y values of your data, and dy= 1-d array containing the uncertainties)
        then you will get plots with the fitted lines, chi-squared value, and the residuals graph.  
    
    
    2.  If you call the function with autoFit(n=number of points), then the function will auto-populate some data, and assume you want boxes to use to move them around and turn the fit on and off for those points.  
    
    
    3.  If you call the function with autoFit(x=1-d vector containing the x values of your data, y=[number of x values, number of trials] array containing the y values of your data)
        then this function will automatically calculate the average y values and uncertainties associated and print out the uncertainties.  
        then you will get plots with the fitted lines, chi-squared value, and the residuals graph.  
    
    4.  If you call the function with autoFit(path="path to csv file") then the function will open the csv file and read in the first column as the x values, 
        the second column as the y values, and the third as the uncertainties in the y values.  
    
    For all of these options, if you want to move the points after inputting them or turn the fit on or off for certain points, use movePoints=True in your call.  
    
    If you want to explore the chi-squared values for different slopes and intercepts like in the manual fitting function, use sliders=True.  
    '''
    
    if len(path)!=0:
        print("using data from: " + path)
        file=open(path)
        lines=file.readlines()
        file.close()
        x=[]
        y=[]
        dy=[]
        for line in lines:
            l=line.split(",")
            x.append(eval(l[0]))
            y.append(eval(l[1]))
            dy.append(eval(l[2]))
        x=np.array(x)
        print("x: ")
        print(x)
        y=np.array(y)
        print("y: ")
        print(y)
        dy=np.array(dy)
        print("uncertainties: ")
        print(dy)
    if len(x)==0:
        x=np.arange(n)
        y=np.arange(n)
        y[0]=1
        dy=np.ones(n)
        movePoints=True
        sliders=True
    if not len(x)==len(y):
        print("The length of x and y have to be the same.")
        return
    x=np.array(x)
    y=np.array(y)
    include=np.ones(len(x), dtype=bool)  # keeps track of if points are included in the fit.  
    #print(include)
    fit=not sliders  #if fit is true then the line automatically snaps to the best fit slope and intercept. 
    if len(dy)==0:
        if (len(y[0]))==0:
            print("You either have to provide either uncertainties or multiple trials in the form of a matrix.")
            return
        else:  #calculate the uncertaintiy and mean automatically if a matrix is given.  
            dy=np.std(y, axis=1)/(len(y[0])**.5)
            y=np.mean(y, axis=1)
            print("Calculated Uncertainties:  ")
            print(dy)
    
    fig, ax=plt.subplots(1,2, figsize=(6,3))

    
    ax[1].set_title("residuals")
    residuals=linear(x,[0, 1])-y
    res=ax[1].errorbar(x,residuals, dy, fmt='.k')
    ax[1].grid(True, which='both')
    plt.show()
    optimal, cov=curve_fit(linearFit,x,y,sigma=dy)
    
    def updateSubmit():  #This is for the boxes.  
        
        #optimal=minimize(chiSquared2, [0,1], args=(x[include],y[include], dy[include], function)).x
        optimal, cov=curve_fit(linearFit,x[include],y[include],sigma=dy[include])
        intercept=optimal[0]  #optimal is the best fit values of the parameters
        slope=optimal[1]
        fx=linear(x,[intercept, slope])
        sigmas=(np.sqrt(np.diag(cov)))  #cov is the covariance matrix which is where we get the uncertainties from. 
        #sigmas are the uncertainties in the slope and intercept.  
        residuals=y-fx
        ax[1].cla() #clear out the graphs.  
        ax[0].cla()
        ax[0].set_title(title)
        ax[0].set_xlabel(xaxis)
        ax[0].set_ylabel(yaxis)

        if fit:
            line,=ax[0].plot(x,fx,     
                         label=r"$\chi^2=$"
                         +str(chiSquared(x[include],y[include], dy[include], function, [intercept, slope]).round(3))
                         +"\n y={}x+{}".format(round(slope, 3), round(intercept, 3))
                         +"\n $\delta_a=${} , $\delta_b=${}".format(round(sigmas[1],2), round(sigmas[0],2)))
        else:
            line,=ax[0].plot(x,fx, 
                         label=r"$\chi^2=$"
                         +str(chiSquared(x[include],y[include], dy[include], function, [intercept, slope]).round(3))
                         +"\n y={}x+{}".format(round(slope, 3), round(intercept, 3)))
        ax[0].legend(prop={'size': 6})
        data=ax[0].errorbar(x,y, dy, fmt='.k')
        res=ax[1].errorbar(x,residuals, dy, fmt='.k')
        ax[1].set_title("residuals")
        ax[1].grid(True, which='both')
        ax[1].set_xlabel(xaxis)
        ax[1].set_ylabel(yaxis)
        fig.canvas.draw_idle()  #redraws canvas. If you don't do this it will not show the updates.  
        plt.tight_layout()  #reorganizes graph.  
    def submit_y(y_i, index=0):
        try:
            y[index]=eval(y_i)
            updateSubmit()
        except:
            return
        
        updateSubmit()
    def submit_x(x_i, index=0):
        try:
            x[index]=eval(x_i)
            updateSubmit()
            return
        except:
            return
    def submit_dy(dy_i, index=0):
        try:
            dy[index]=eval(dy_i)
            updateSubmit()
        except:
            return 
        
    def fitTo(include_in_fit, index=0):
        try:
            include[index]=include_in_fit
            updateSubmit()
        except:
            return
        
    def update(intercept=optimal[0],slope=optimal[1], fit=True):  #this is for the sliders.  
        if fit:
            #optimal=minimize(chiSquared2, [0,1], args=(x[include],y[include], dy[include], function)).x
            optimal, cov=curve_fit(linearFit,x[include],y[include],sigma=dy[include])
            #print(minimize(chiSquared2, [0,1], args=(x[include],y[include], dy[include], function)))
            intercept=optimal[0]
            slope=optimal[1]
            sigmas=(np.sqrt(np.diag(cov)))
        fx=linear(x,[intercept, slope])
        residuals=y-fx
        ax[1].cla()
        ax[0].cla()
        ax[0].set_title(title)
        ax[0].set_xlabel(xaxis)
        ax[0].set_ylabel(yaxis)
        if fit:
            line,=ax[0].plot(x,fx, 
                         label=r"$\chi^2=$"
                         +str(chiSquared(x[include],y[include], dy[include], function, [intercept, slope]).round(3))
                         +"\n y={}x+{}".format(round(slope, 3), round(intercept, 3))
                         +"\n $\delta_a=${} , $\delta_b=${}".format(round(sigmas[1],2), round(sigmas[0],2)))
        else:
            line,=ax[0].plot(x,fx, 
                         label=r"$\chi^2=$"
                         +str(chiSquared(x[include],y[include], dy[include], function, [intercept, slope]).round(3))
                         +"\n y={}x+{}".format(round(slope, 3), round(intercept, 3)))
        ax[0].legend(prop={'size': 6})
        data=ax[0].errorbar(x,y, dy, fmt='.k')
        res=ax[1].errorbar(x,residuals, dy, fmt='.k')
        ax[1].set_title("residuals")
        ax[1].grid(True, which='both')
        ax[1].set_xlabel(xaxis)
        ax[1].set_ylabel(yaxis)
        fig.canvas.draw_idle()
        plt.tight_layout()
    optimal, cov=curve_fit(linearFit,x[include],y[include],sigma=dy[include]) 
    i=optimal[0]
    sl=optimal[1]
    #minimum and maximum slope and intercept that we allow on the sliders.  
    minInt=min((i*.1)-1, i*10+1)
    maxInt=max((i*.1)-1, i*10+1)
    minSlo=min((sl*.1)-1, sl*10+1)
    maxSlo=max((sl*.1)-1, sl*10+1)
    interact(update, intercept=(minInt, maxInt, (maxInt-minInt)/500), slope=(minSlo,maxSlo, (maxSlo-minSlo)/500), fit=True);
    if movePoints:
        for value in range(len(y)):
            print("datapoint: {}".format(value+1))
            interact(submit_x, x_i=str(x[value]), index=fixed(value))
            interact(submit_y, y_i=str(y[value]), index=fixed(value))
            interact(submit_dy, dy_i=str(dy[value]), index=fixed(value))
            interact(fitTo, include_in_fit=True, index=fixed(value))
    update()


In [ ]:
#autoFit(path="example.csv",  movePoints=True)

Developed by Rebeckah Fussell for Cornell Physics Labs.

In [ ]:
def standard_deviation(data):
    '''
    Takes in a one-dimensional numpy array argument, and returns the standard deviation of the dataset. 
    '''
    N = len(data)
    return np.sqrt(np.sum((data - np.mean(data))**2)/(N-1))

def standard_unc_of_mean(data):
    '''
    Takes in a one-dimensional numpy array argument, and returns the standard uncertainty of the mean of the dataset. 
    '''
    N = len(data)
    return standard_deviation(data) / np.sqrt(N)
def t_prime(A, dA, B, dB):
    '''
    Returns the value of t prime. Requires 4 arguments in this order: measurement A, uncertainty of measurement A, measurent B, uncertainty of measurement B.
    ''' 
    return abs(A - B) / np.sqrt(dA**2 + dB**2)






Developed by Natasha Holmes for Cornell Physics Labs.

In [ ]:
def dsum(x, dx, y, dy):
    return np.sqrt(dx**2+dy**2)

def dsub(x, dx, y, dy):
    return np.sqrt(dx**2+dy**2)

def dmult(x, dx, y, dy):
    return x*y*np.sqrt((dx/x)**2+(dy/y)**2)

def ddiv(x, dx, y, dy): 
    return x/y*np.sqrt((dx/x)**2+(dy/y)**2)

def dpwr(x, dx, n):
    return n*dx*(x**n)/x

def dln(x, dx):
    return dx/x